In [1]:
!pip install meshio
!pip install gcsfs

In [2]:
import numpy as np
import matplotlib.pyplot as plt 
import xarray as xr
import meshio
import gcsfs

In [4]:
projname = 'ldeo-glaciology'
bucket = 'ldeo-glaciology/elmer_janie/output_vals/'

fs = gcsfs.GCSFileSystem(project=projname)
filelist = fs.ls(bucket)


#info about mesh grid (set by user in model - mesh.grd file)
xgrid_cells = 1000
ygrid_cells = 30

xgrid_pts = xgrid_cells + 1
ygrid_pts = ygrid_cells + 1

xnodes = np.array(range(xgrid_pts)) #ynodes 
ynodes = np.array(range(ygrid_pts))  # xnodes
tsteps = np.array(range(len(filelist)))  #number of time steps

timestep = [] #list of data arrays
for path in filelist:
    t_val = filelist.index(path)
    print(path)
    print(t_val)
    gcs_file = fs.get(path,'out.vtu') #load file
    mesh = meshio.read('out.vtu', file_format= 'vtu') #mesh the file
    
    velx = mesh.point_data['velocity'][:,0].reshape((ygrid_pts,xgrid_pts)) #reshape the mesh coordinates
    vely = mesh.point_data['velocity'][:,1].reshape((ygrid_pts,xgrid_pts))

    xdim = mesh.points[:,0].reshape((ygrid_pts,xgrid_pts))
    ydim = mesh.points[:,1].reshape((ygrid_pts,xgrid_pts))
    
    tempdataset = xr.Dataset(data_vars = {'vel_x' : (('ynode','xnode'), velx) ,'vel_y' :(('ynode','xnode'), vely)},
                      coords = {'yvals': (('ynode','xnode'),ydim),'xvals': (('ynode','xnode'),xdim)})
    timestep.append(tempdataset)

datacomb = xr.concat(timestep, dim='t')

ldeo-glaciology/elmer_janie/output_vals/Output0001.vtu
0
ldeo-glaciology/elmer_janie/output_vals/Output0002.vtu
1
ldeo-glaciology/elmer_janie/output_vals/Output0003.vtu
2
ldeo-glaciology/elmer_janie/output_vals/Output0004.vtu
3
ldeo-glaciology/elmer_janie/output_vals/Output0005.vtu
4
ldeo-glaciology/elmer_janie/output_vals/Output0006.vtu
5
ldeo-glaciology/elmer_janie/output_vals/Output0007.vtu
6
ldeo-glaciology/elmer_janie/output_vals/Output0008.vtu
7
ldeo-glaciology/elmer_janie/output_vals/Output0009.vtu
8
ldeo-glaciology/elmer_janie/output_vals/Output0010.vtu
9
ldeo-glaciology/elmer_janie/output_vals/Output0011.vtu
10
ldeo-glaciology/elmer_janie/output_vals/Output0012.vtu
11
ldeo-glaciology/elmer_janie/output_vals/Output0013.vtu
12
ldeo-glaciology/elmer_janie/output_vals/Output0014.vtu
13
ldeo-glaciology/elmer_janie/output_vals/Output0015.vtu
14
ldeo-glaciology/elmer_janie/output_vals/Output0016.vtu
15
ldeo-glaciology/elmer_janie/output_vals/Output0017.vtu
16
ldeo-glaciology/elmer_ja